# COMBINED RADIOMIC FEATURES

## Installation and importation or third party packages,libraries and dependencies

##### Before training the separate the models, One needs to ensure that the necessary libraries are packages are installed. Pyradiomics was used to extract features from the functional Magnetic Resonance Imgaging (fMRI) files. Scikit-learn libraries were used to create the models. Setting various parameters to yield results and performing performance metrics. 

In [ ]:
!pip install Pyradiomics

In [ ]:
import os
import SimpleITK as sitk
import radiomics
from radiomics import featureextractor

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

2023-05-25 00:06:36.896401: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Feature Extraction

In [ ]:
# To load the ROI mask of each fMRI image
roi_path = "image43_mask2.nii"
roi_image = sitk.ReadImage(roi_path)

# Initiating the the Pyradiomics extractor
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor()


# Extracting the features from the ROI mask to a dictionary
result = extractor.execute(roi_image,'image43_mask2.nii' ) 



In [ ]:
# Concatenating the results and printing the dictionary in an orderly manner
print ("result type:") , type(result)
print ("")
print ("calculated features")
for key, value in result.items():
    print ("\t", key, ":", value)

In [ ]:
df = pd.DataFrame.from_dict(result, orient='index', columns=['Value'])

print (df)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


print (df)

In [ ]:
df.index = df.index.str.replace('original_', '')

In [ ]:
df.to_csv('output43.csv')

## Loading the extracted radiomics feature data

In [ ]:
# Loading the data from the CSV file
data = pd.read_csv('Combined_radiomics.csv')

# Excluding the first column (patient ID)
data = data.iloc[:, 1:]

# Separating the features (X) and the labels (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 42)




## Normalisation (min-max)

In [ ]:
# Apply min-max normalization to scale the features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



## Feature selection and building the SVM model with the combined radiomic features 

In [ ]:
# Create an SVM classifier
svm = SVC(kernel='linear')

# Train the SVM classifier
svm.fit(X_train, y_train)

# Get the feature importances (coefficients)
feature_importances = abs(svm.coef_[0])

# Sort the features based on their importance in descending order
sorted_indices = feature_importances.argsort()[::-1]

# Select the top-k features. In this study, the top 50 features were selected. 
k = 50

selected_features = sorted_indices[:k]

# Extract the selected features from the training and testing sets
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]

# Train the SVM classifier on the selected features
svm_selected = SVC()
svm_selected.fit(X_train_selected, y_train)

# Predict the labels for the test set
y_pred = svm_selected.predict(X_test_selected)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)


print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)

In [ ]:

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d", cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred)

# Generate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve (AUC = {:.2f})'.format(auc_score))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

## Artificial Neural Network (ANN) with the combined radiomic features

In [ ]:
# Load the data from the CSV file
data2 = pd.read_csv('Combined_radiomics.csv')

# Exclude the first column (patient ID)
datareal = data2.iloc[:, 1:]


# Separate the features (X) and the labels (y)
X2 = datareal.iloc[:, :-1].values
y2 = datareal.iloc[:, -1].values

# Split the data into training and testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size= 0.2, random_state=40)

# Apply min-max normalization to scale the features
scaler = MinMaxScaler()
Xreal_train = scaler.fit_transform(X2_train)
Xreal_test = scaler.transform(X2_test)

# Perform feature selection using RFE
estimator = SVC(kernel='linear')
rfe = RFE(estimator, n_features_to_select= 50)
X_train_selected = rfe.fit_transform(Xreal_train, y2_train)
X_test_selected = rfe.transform(Xreal_test)

# Define the ANN model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train_selected, y2_train, epochs=50, batch_size=32)

# Predict the labels for the test set
y_pred_prob = model.predict(X_test_selected)
y_pred2 = np.round(y_pred_prob)

# Calculate the confusion matrix
cm2 = confusion_matrix(y2_test, y_pred2)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm2, annot=True, cmap="Blues", fmt="d", cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Calculate the AUC score
auc_score2 = roc_auc_score(y2_test, y_pred_prob)

# Generate the ROC curve
fpr2, tpr2, thresholds = roc_curve(y2_test, y_pred_prob)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr2, tpr2, label='ROC Curve (AUC = {:.2f})'.format(auc_score2))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

# Calculate other performance metrics
accuracy2 = (cm2[0, 0] + cm2[1, 1]) / np.sum(cm2)
precision2 = cm2[1, 1] / (cm2[1, 1] + cm2[0, 1])
recall2 = cm2[1, 1] / (cm2[1, 1] + cm2[1, 0])
f1_score2 = 2 * (precision2 * recall2) / (precision2 + recall2)

print('Accuracy:', accuracy2)
print('Precision:', precision2)
print('Recall:', recall2)
print('F1-Score:', f1_score2)


# SHAPE RADIOMIC FEATURES

### Two supervised learning algorithms (SVM and ANN) were trained on only shape features and without feature selection.

## SVM

In [ ]:
# Load the data from the CSV file
data3 = pd.read_csv('Shape radiomic features.csv')

# Exclude the first column (patient ID)
datasvm2 = data3.iloc[:, 1:]

# Separate the features (X) and the labels (y)
X3 = datasvm2.iloc[:, :-1].values
y3 = datasvm2.iloc[:, -1].values

# Split the data into training and testing sets
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state= 40)

# Apply min-max normalization to scale the features
scaler3 = MinMaxScaler()
X3_train = scaler3.fit_transform(X3_train)
X3_test = scaler3.transform(X3_test)

# Create an SVM classifier
svm3 = SVC(kernel='linear')

# Train the SVM classifier
svm3.fit(X3_train, y3_train)


# Predict the labels for the test set
y_pred3 = svm3.predict(X3_test)

# Calculate the accuracy of the model
accuracy3 = accuracy_score(y3_test, y_pred3)

precision3 = precision_score(y3_test, y_pred3)
recall3 = recall_score(y3_test, y_pred3)
f13 = f1_score(y3_test, y_pred3)
roc_auc3 = roc_auc_score(y3_test, y_pred3)


print("Accuracy:", accuracy3)
print("Precision:", precision3)
print("Recall:", recall3)
print("F1 Score:", f13)
print("ROC AUC Score:", roc_auc3)

In [ ]:
# Calculate the confusion matrix
cm3 = confusion_matrix(y3_test, y_pred3)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm3, annot=True, cmap="Blues", fmt="d", cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Calculate the AUC score
auc_score3 = roc_auc_score(y3_test, y_pred3)

# Generate the ROC curve
fpr3, tpr3, thresholds = roc_curve(y3_test, y_pred3)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve (AUC = {:.2f})'.format(auc_score3))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

## ARTIFICIAL NEURAL NETWORK (ANN)

In [ ]:
# Load the data from the CSV file
data4 = pd.read_csv('Shape radiomic features.csv')

# Exclude the first column (patient ID)
datareal4 = data4.iloc[:, 1:]


# Separate the features (X) and the labels (y)
X4 = datareal4.iloc[:, :-1].values
y4 = datareal4.iloc[:, -1].values

# Split the data into training and testing sets
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size= 0.2, random_state=42)

# Apply min-max normalization to scale the features
scaler4 = MinMaxScaler()
Xreal_train4 = scaler4.fit_transform(X4_train)
Xreal_test4 = scaler4.transform(X4_test)


# Define the ANN model architecture
model4 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model4.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model4.fit(Xreal_train4, y4_train, epochs=50, batch_size=32)

# Predict the labels for the test set
y_pred_prob4 = model4.predict(Xreal_test4)
y_pred4 = np.round(y_pred_prob4)

# Calculate the confusion matrix
cm4 = confusion_matrix(y4_test, y_pred4)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm4, annot=True, cmap="Blues", fmt="d", cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Calculate the AUC score
auc_score4 = roc_auc_score(y4_test, y_pred_prob4)

# Generate the ROC curve
fpr4, tpr4, thresholds = roc_curve(y4_test, y_pred_prob4)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr2, tpr2, label='ROC Curve (AUC = {:.2f})'.format(auc_score4))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

# Calculate other performance metrics
accuracy4 = (cm4[0, 0] + cm4[1, 1]) / np.sum(cm4)
precision4 = cm4[1, 1] / (cm4[1, 1] + cm4[0, 1])
recall4 = cm4[1, 1] / (cm4[1, 1] + cm4[1, 0])
f1_score4 = 2 * (precision4 * recall4) / (precision4 + recall4)

print('Accuracy:', accuracy4)
print('Precision:', precision4)
print('Recall:', recall4)
print('F1-Score:', f1_score4)
